In [1]:
# import numpy as np
# import pandas as pd
# from src.channel import Channel
# from src.encoder import PolarEncoder
# from src.decoder import SCLDecoder


# rank = pd.read_csv("rank.csv")

# rank

In [2]:
# # Длина кода
# N = 512

# # Скорость кода
# R = 1 / 2

# # Количество информационных бит
# K = int(R * N)

# reliability = np.array(rank["Q"])
# # --- выбор замороженных позиций ---
# freeze_positions = []
# for idx in reliability:
#     if idx < N:
#         freeze_positions.append(idx)
#         if len(freeze_positions) == K:
#             break

# freeze_positions = np.sort(np.array(freeze_positions, dtype=int))
# info_positions = np.sort(np.setdiff1d(np.arange(N), freeze_positions))

# print("K =", K)
# print("info_positions (len={}):".format(len(info_positions)), info_positions)
# print("freeze_positions (len={}):".format(len(freeze_positions)), freeze_positions)

In [3]:
# # Генерируем сообщение, состощее из случайной последовательности 0 и 1
# message = np.random.randint(0, 2, K)
# # message = np.array([1, 0, 1, 0])
# print(f"message = {message}")

# # Вектор u
# u = np.zeros(N, dtype=int)
# print(f"u = {u}")

# # Вставляем значения из message в позиции info_positions
# u[info_positions] = message
# print(f"u = {u}")

In [4]:
# def tensor_power(F, n):
#     """Возвращает F^⊗n (кронекерово произведение n раз)"""
#     if n == 0:
#         return np.array([[1]])
#     elif n == 1:
#         return F
#     else:
#         # Рекурсивно: F^⊗n = F ⊗ (F^⊗(n-1))
#         return np.kron(F, tensor_power(F, n - 1))


# # Применяем полярное преобразование
# F = np.array([[1, 0], [1, 1]])
# n = int(np.log2(N))
# G_N = tensor_power(F, n)
# print(f"F^⊗{n} (размер {G_N.shape[0]}x{G_N.shape[1]}):")
# print(f"G_N = {G_N}")
# x = (u @ G_N) % 2
# print(f"x = {x}")

In [5]:
# def polar_encode(u):
#     """
#     Быстрое преобразование для полярного кодирования.
#     u — вектор длины N (N должно быть степени двойки).
#     Возвращает кодовое слово x.
#     """
#     N = len(u)
#     x = u.copy()

#     stage = 1
#     while stage < N:
#         half = stage
#         step = 2 * stage
#         for i in range(0, N, step):
#             for j in range(half):
#                 x[i + j] ^= x[i + j + half]  # XOR комбинация
#         stage *= 2

#     return x


# x = polar_encode(u)
# print(f"x = {x}")

In [6]:
# def bpsk_mod(x):
#     return 1 - 2 * x  # 0->+1, 1->-1


# s = bpsk_mod(x)
# print(f"x = {s}")

In [7]:
# # Канал AWGN
# sigma2 = 0.5
# noise = np.random.normal(0, np.sqrt(sigma2), N)
# print(f"noise = {noise}")
# y = s + noise
# print(f"y = {y}")

# # LLR
# LLR = 2 * y / sigma2
# print(f"LLR = {LLR}")

In [8]:
# # LLR = [-0.3, -1.2, 0.7, -0.8, -1.1, 0.9, -1.6, -0.5]
# # print(LLR)
# d_LLR, b, path_metrics = decompose(LLR)
# print(d_LLR)
# print(recursive_to_array(d_LLR, N))
# print()
# u_hat = []
# for el in path_metrics:
#     u_hat.append(el["bit"])
#     print(f"{el["id"]}: pm = {el["pm"]}, bit = {el["bit"]}")

# u_hat = np.array(u_hat)
# print(u_hat)
# decoded = u_hat[info_positions]
# print(message)
# print(decoded)
# if np.array_equal(decoded, message):
#     print("УСПЕШНОЕ ДЕКОДИРОВАНИЕ")
# else:
#     print("ОШИБКА ДЕКОДИРОВАНИЯ")
# # # f_LLR = find(d_LLR, 9, 2)
# # # print(f_LLR)
# # for i in range(N):
# #     print(find(d_LLR, int(np.log2(N)), i), i)

In [9]:
# # SCL
# scl_L = 2
# paths = [{"path": [], "pm": 0}]
# for el in path_metrics:
#     if el["id"] in freeze_positions:
#         print("f")
#         for p in paths:
#             p["path"].append(el["bit"])
#             if el["pm"] < 0:
#                 p["pm"] += abs(el["pm"])
#             print(p)
#         print(paths)
#     else:
#         print("i")
#         new_paths = []
#         for i in range(len(paths)):
#             path = paths[i].copy()
#             for b in [0, 1]:
#                 path["path"].append(b)
#                 if el["bit"] != b:
#                     path["pm"] += abs(el["pm"])
#                 new_paths.append(path)
#         new_paths.sort(key=lambda x: x["pm"])
#         paths = new_paths[:scl_L]
# best_path = min(paths, key=lambda x: x["pm"])

In [10]:
# _, _, _, p = calc_pm([0] * N, LLR)
# print(p)

In [11]:
import numpy as np
import pandas as pd
from src.channel import Channel
from src.encoder import PolarEncoder
from src.decoder import SCLDecoder

# Длина кода
N = 512

# Скорость кода
R = 1 / 2

# Количество информационных бит
K = int(R * N)

# Длина списка scl декодера
L = 16

# Генерируем сообщение, состощее из случайной последовательности 0 и 1
message = np.random.randint(0, 2, K)
# message = np.array([0, 0, 1, 1])
# message = np.array([1, 0, 1, 0])
# message = np.array([1, 1, 1, 1,])
print(f"message = {message}")

polar_encoder = PolarEncoder(N, R, K, "rank.csv")
info_positions, freeze_positions = polar_encoder.set_info_and_freeze_positions()
u = polar_encoder.get_u_vector(message)
x = polar_encoder.encode(u)
print(x)
s = polar_encoder.bpsk_mod(x)

channel = Channel(0.8, N)
y = channel.effect(s)
LLR = channel.calc_LLR(y)
# LLR = [-1.7, -0.63, -1.3, 1.1, 0.39, -0.09, -1.22, -0.77]
# LLR = [-0.3, -1.2, 0.7, -0.8, -1.1, 0.9, -1.6, -0.5]

scl_decoder = SCLDecoder(N, R, K, L, freeze_positions, info_positions)
u_hat, decoded = scl_decoder.sc_decode(LLR, message)

message = [0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1
 1 1 1 0 1 1 0 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0
 0 1 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1
 0 1 1 1 1 1 0 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 0 0
 0 1 0 1 0 0 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0
 1 1 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1
 1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0]
u = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 1 0
 1 1 0 0 0 1 0 1

In [ ]:
best_path = scl_decoder.scl_decode(LLR, message)
print(best_path)
# print(recursive_to_array(d_LLR, self.N))
# print()
u_hat = best_path["path"].copy()
# for el in path_metrics:
#     u_hat.append(el["bit"])
#     # print(f"{el["id"]}: pm = {el["pm"]}, bit = {el["bit"]}")

u_hat = np.array(u_hat)
# print(u_hat)
decoded = u_hat[info_positions]
print(f"message = {message}")
print(f"decoded = {decoded}")
if np.array_equal(decoded, message):
    print("=" * 100)
    print("УСПЕШНОЕ ДЕКОДИРОВАНИЕ")
    print("=" * 100)
else:
    print("=" * 100)
    print("ОШИБКА ДЕКОДИРОВАНИЯ")
    print("=" * 100)

{'path': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

: 